In [42]:
import sys; sys.path.append('..')
def REQUIRES(x): assert(x)
ENSURES = REQUIRES
from bending_validation import *
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
from typing import NamedTuple
from bending_validation import suppress_stdout as so

In [43]:
pts = np.array([[-500,500,0], [500,500,0],[500,-500,0],[-500,-500,0]])
edges = np.array([[0,1],[1,2],[2,3],[3,0]])  

In [44]:
sq = elastic_rods.RodLinkage(pts, edges, 5)

In [45]:
driver=sq.centralJoint()
sq.setMaterial(elastic_rods.RodMaterial('rectangle', 4.0e10 / 1e6, 0.3, [50, 50]))

elastic_rods.restlen_solve(sq)

0	7.42756e-21	8.8661e-08	8.8661e-08	1	0
1	6.16298e-22	1.66533e-08	1.66533e-08	1	0


In [46]:
driver # the joint index i.e. as there are 4 joints, 0th index is the 1st joint

0

In [47]:
sq.dofOffsetForJoint(driver) # the exact index of this joint in the DoFs array

36

In [48]:
sq.getDoFs()[36:42] # x, y, z coordinates + some other parameters ?

array([-5.00000000e+02,  5.00000000e+02,  0.00000000e+00, -1.18316196e-63,
        9.22969858e-32, -2.75803504e-48])

In [49]:
sq.dofOffsetForJoint(1), sq.dofOffsetForJoint(2), sq.dofOffsetForJoint(3)

(45, 54, 63)

In [50]:
sq.getDoFs()[45:51], sq.getDoFs()[54:60], sq.getDoFs()[63:69]

(array([ 5.00000000e+02,  5.00000000e+02,  0.00000000e+00,  4.13673768e-64,
        -4.38087762e-32,  1.29269371e-48]),
 array([ 5.00000000e+02, -5.00000000e+02,  0.00000000e+00,  1.01288780e-64,
        -1.44377466e-32,  3.21779887e-48]),
 array([-5.00000000e+02, -5.00000000e+02,  0.00000000e+00, -2.36850850e-64,
         5.16550102e-32, -2.06766256e-48]))

In [51]:
sq.dofOffsetForJoint(4)

IndexError: vector::_M_range_check: __n (which is 4) >= this->size() (which is 4)

In [52]:
jdo = sq.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
elastic_rods.compute_equilibrium(sq, fixedVars=fixedVars)

0	6.16298e-22	1.57009e-08	1.57009e-08	0	0


In [53]:
# Original View
view = linkage_vis.LinkageViewer(sq)
view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [54]:
jointDoFIndices = sq.jointDoFIndices()

In [55]:
np.array(jointDoFIndices)

array([36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71])

In [56]:
len(jointDoFIndices)

36

In [57]:
dofs = sq.getDoFs()
dofs[jointDoFIndices]

array([-500.        ,  500.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    1.57079633,  250.        ,
        250.        ,  500.        ,  500.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    1.57079633,
        250.        ,  250.        ,  500.        , -500.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          1.57079633,  250.        ,  250.        , -500.        ,
       -500.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    1.57079633,  250.        ,  250.        ])

In [58]:
topLeft = jointDoFIndices[0:9]
topRight = jointDoFIndices[9:18]
bottomRight = jointDoFIndices[18:27]
bottomLeft = jointDoFIndices[27:36]

In [59]:
sq.getDoFs()[topLeft], sq.getDoFs()[bottomLeft], sq.getDoFs()[topRight], sq.getDoFs()[bottomRight]
# go up the call stack

(array([-500.        ,  500.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    1.57079633,  250.        ,
         250.        ]),
 array([-500.        , -500.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    1.57079633,  250.        ,
         250.        ]),
 array([500.        , 500.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.57079633, 250.        ,
        250.        ]),
 array([ 500.        , -500.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    1.57079633,  250.        ,
         250.        ]))

In [60]:
f = [0] * sq.numDoF()
f[bottomLeft[1]] = -5000

rigidMotionVars = [] # bottomLeft + topLeft

In [61]:
with so(): elastic_rods.compute_equilibrium(sq, externalForces=f, fixedVars=topRight)
    # step up

In [62]:
deformedView = linkage_vis.LinkageViewer(sq)
deformedView.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [63]:
sq.getDoFs()[topRight], sq.getDoFs()[bottomRight]

(array([500.        , 500.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.57079633, 250.        ,
        250.        ]),
 array([ 5.00052832e+02, -4.99999996e+02, -3.56948679e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.64902190e+00,  2.49999999e+02,
         2.49999992e+02]))

In [64]:
sq.getDoFs()[topLeft], sq.getDoFs()[bottomLeft]

(array([-4.96211908e+02,  4.21823703e+02, -1.03583689e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.69076240e+00,  2.50012501e+02,
         2.50001496e+02]),
 array([-4.96882780e+02, -5.78226074e+02,  2.06206259e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.64977325e+00,  2.50012501e+02,
         2.49999992e+02]))

In [65]:
deformedViewXX = linkage_vis.LinkageViewer(sq)
deformedViewXX.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [66]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, sq, opts, fv):
    opts.gradTol = 1e-4
    return elastic_rods.compute_equilibrium(sq, tgtAngle, options=opts, fixedVars=fv)
with so(): open_linkage(sq, driver, np.deg2rad(60) - sq.averageJointAngle, 40, deformedViewXX, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, useTargetAngleConstraint=True);

In [67]:
sq.getDoFs()[topLeft], sq.getDoFs()[topRight], sq.getDoFs()[bottomLeft], sq.getDoFs()[bottomRight]

(array([-4.96211908e+02,  4.21823703e+02, -1.03583689e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.04719755e+00,  2.50000000e+02,
         2.50000000e+02]),
 array([4.83357511e+02, 2.20717378e+02, 8.24936825e-17, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.04719755e+00, 2.50000000e+02,
        2.50000000e+02]),
 array([-1.80590385e+02, -5.27061461e+02,  1.68866574e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.04719755e+00,  2.50000000e+02,
         2.50000000e+02]),
 array([ 7.98979033e+02, -7.28167786e+02, -2.57134115e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.04719755e+00,  2.50000000e+02,
         2.50000000e+02]))

In [68]:
f = [0] * sq.numDoF()
f[topRight[1]] = -100
with so(): elastic_rods.compute_equilibrium(sq, f, fixedVars=topLeft[0:3])

In [69]:
deformedViewX = linkage_vis.LinkageViewer(sq)
deformedViewX.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [70]:
sq.getDoFs()[topLeft], sq.getDoFs()[topRight], sq.getDoFs()[bottomLeft], sq.getDoFs()[bottomRight]

(array([-4.96211908e+02,  4.21823703e+02, -1.03583689e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.38514851e-01,  2.50000520e+02,
         2.49999688e+02]),
 array([ 8.20626200e+01, -3.94059718e+02,  1.48559108e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.38461559e-01,  2.49999480e+02,
         2.49999688e+02]),
 array([-3.58418779e+01, -4.65867429e+02, -9.65690550e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.38568179e-01,  2.50000520e+02,
         2.50000514e+02]),
 array([ 5.42479756e+02, -1.28172149e+03, -8.20220779e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.38514887e-01,  2.49999480e+02,
         2.50000514e+02]))